In [26]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
import pytesseract
import imutils
from transform import four_point_transform
from skimage.filters import threshold_local
import numpy as np

pytesseract.pytesseract.tesseract_cmd = 'C:\\Program Files\\Tesseract-OCR\\tesseract.exe'

image = cv2.imread('IMG_1180.jpg')
#image = cv2.resize(img,(600,600))
min_h = int((12.5*image.shape[0])/100)
ratio = image.shape[0] / min_h
print(image.shape)
orig = image.copy()
image = imutils.resize(image, height = min_h )

gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
gray = cv2.GaussianBlur(gray, (5, 5), 0)
edged = cv2.Canny(gray, 75, 200)
#cv2.imshow('ada',edged)
#cv2.waitKey(0)
#cv2.destroyAllWindows()

# print(pytesseract.image_to_string(gray))

(4032, 3024, 3)


In [27]:
cnts = cv2.findContours(edged.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
cnts = imutils.grab_contours(cnts)
cnts = sorted(cnts, key = cv2.contourArea, reverse = True)[:5]

for c in cnts:
    peri = cv2.arcLength(c, True)
    approx = cv2.approxPolyDP(c, 0.02 * peri, True)
    #cv2.drawContours(image, [approx], -1, (0, 255, 0), 2)
    if len(approx) == 4:
        screenCnt = approx
        break
# show the contour (outline) of the piece of paper
print("contours: ")
cv2.drawContours(image, [screenCnt], -1, (0, 255, 0), 2)


contours: 


array([[[ 18,  22,  19],
        [ 20,  23,  21],
        [ 20,  22,  20],
        ...,
        [ 58,  64,  69],
        [124, 134, 142],
        [192, 203, 211]],

       [[ 18,  22,  19],
        [ 19,  23,  20],
        [ 20,  22,  20],
        ...,
        [ 81,  87,  92],
        [115, 126, 134],
        [153, 164, 172]],

       [[ 20,  22,  20],
        [ 22,  24,  22],
        [ 20,  20,  20],
        ...,
        [ 86,  92,  97],
        [123, 133, 142],
        [162, 172, 180]],

       ...,

       [[ 13,  19,  18],
        [ 12,  17,  18],
        [  9,  15,  14],
        ...,
        [ 18,  24,  22],
        [ 20,  25,  24],
        [ 19,  25,  24]],

       [[ 11,  17,  15],
        [ 11,  16,  16],
        [  9,  16,  15],
        ...,
        [ 18,  25,  23],
        [ 19,  25,  23],
        [ 19,  25,  24]],

       [[ 11,  17,  14],
        [ 10,  16,  14],
        [  8,  17,  15],
        ...,
        [ 13,  22,  19],
        [ 17,  26,  23],
        [ 19,  27,  24]]

In [28]:
# apply the four point transform to obtain a top-down
# view of the original image
warped = four_point_transform(orig, screenCnt.reshape(4, 2) * ratio)
# convert the warped image to grayscale, then threshold it
# to give it that 'black and white' paper effect
warped = cv2.cvtColor(warped, cv2.COLOR_BGR2GRAY)
T = threshold_local(warped, 11, offset = 10, method = "gaussian")
warped = (warped > T).astype("uint8") * 255

In [29]:
#cv2.namedWindow("output", cv2.WINDOW_NORMAL)                             
imS = cv2.resize(warped, (900,500))

#contrast_warped = cv2.normalize(warped,None,alpha=0,beta=255,norm_type=cv2.NORM_MINMAX)

#ret,threshed = cv2.threshold(contrast_warped,120,255,cv2.THRESH_BINARY_INV)

#plt.imshow(contrast_warped,cmap='gray')
#plt.show()
cv2.imwrite('D:/BCP/Bill1180.jpg',warped)
#cv2.imshow("output",constrast_warped)                          
#cv2.waitKey(0) 
#cv2.destroyAllWindows()

True

In [21]:
#print(pytesseract.image_to_string(warped))
#cv2.imwrite('D:/BCP/Test.jpg',warped)